In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install transformers datasets evaluate accelerate
!pip3 install torch torchvision torchaudio --upgrade --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
torch.__version__ 

In [ ]:
!bash run_modded.sh